In [47]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement

In [48]:
town_level_df = pd.read_csv('data/overall_supply_and_estimated_demand_for_child_care_by_age_and_by_town.csv')
town_level_funded = pd.read_csv('data/town_level_children_funded_demand_and_supply.csv')

In [49]:
geo_df = gpd.read_file('zip:////Users/kylemagida/code/skylight/data-visualization/src/data_integration/unmet_needs/census_data/data/cousub/carto/cb_2018_09_cousub_500k.zip')

In [50]:
geo_df.head()

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,09,015,09190,00213400,0600000US0901509190,0901509190,Brooklyn,43,75454825,282000,"POLYGON ((-72.01801 41.78263, -72.01449 41.804..."
1,09,013,72090,00212349,0600000US0901372090,0901372090,Stafford,43,150316815,1868958,"POLYGON ((-72.40675 41.95716, -72.40442 41.979..."
2,09,013,77830,00213519,0600000US0901377830,0901377830,Union,43,74720104,2514328,"POLYGON ((-72.22328 41.98167, -72.21642 41.996..."
3,09,007,14300,00213407,0600000US0900714300,0900714300,Chester,43,41573118,2010913,"POLYGON ((-72.53878 41.42149, -72.52416 41.424..."
4,09,009,53890,00213477,0600000US0900953890,0900953890,North Branford,43,64123377,4939189,"POLYGON ((-72.82380 41.31843, -72.82325 41.321..."


In [18]:
geo_df.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [51]:
merged_df = geo_df.merge(town_level_df, how='left', left_on='NAME', right_on='City')

In [68]:
# https://gis.stackexchange.com/questions/300747/how-to-convert-geoseries-polygons-to-lists-of-coordinates
def coord_lister(geom):
    coords = list(geom.exterior.coords)
    return [list(x) for x in list(coords)]

In [70]:
merged_df['geometry'].apply(coord_lister)

AttributeError: 'MultiPolygon' object has no attribute 'exterior'

In [33]:
engine = create_engine('sqlite:///file.db', echo=True)

In [ ]:
# https://stackoverflow.com/questions/38361336/write-geodataframe-into-sql-database
def create_wkt_element(geom):
    return WKTElement(geom.wkt, srid =4269)

In [ ]:
merged_df['shape'] = merged_df['geometry'].apply(create_wkt_element)

In [34]:
sub_table = merged_df[['shape','Available_Spaces_for_Children_Supply_Preschoolers','NAME']]

In [35]:
merged_df['shape'] = merged_df['geometry'].apply(create_wkt_element)

In [36]:
sub_table = merged_df[['shape','Available_Spaces_for_Children_Supply_Preschoolers','NAME']]

In [46]:
geo_df.to_postgis('geo1', con=engine)

2021-02-22 15:38:48,016 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-22 15:38:48,019 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("geo1")
2021-02-22 15:38:48,020 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 15:38:48,021 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("geo1")
2021-02-22 15:38:48,021 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 15:38:48,033 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE geo1 (
	"STATEFP" TEXT, 
	"COUNTYFP" TEXT, 
	"COUSUBFP" TEXT, 
	"COUSUBNS" TEXT, 
	"AFFGEOID" TEXT, 
	"GEOID" TEXT, 
	"NAME" TEXT, 
	"LSAD" TEXT, 
	"ALAND" BIGINT, 
	"AWATER" BIGINT, 
	geometry geometry(GEOMETRY,4269)
)


2021-02-22 15:38:48,034 INFO sqlalchemy.engine.base.Engine ()
2021-02-22 15:38:48,035 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) near "GEOMETRY": syntax error
[SQL: 
CREATE TABLE geo1 (
	"STATEFP" TEXT, 
	"COUNTYFP" TEXT, 
	"COUSUBFP" TEXT, 
	"COUSUBNS" TEXT, 
	"AFFGEOID" TEXT, 
	"GEOID" TEXT, 
	"NAME" TEXT, 
	"LSAD" TEXT, 
	"ALAND" BIGINT, 
	"AWATER" BIGINT, 
	geometry geometry(GEOMETRY,4269)
)

]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [42]:
sub_table[:1].to_postgis('geo', engine, if_exists='append', index=False, 
                         dtype={'geom': Geometry('POINT', srid= 4269)})

AttributeError: 'DataFrame' object has no attribute 'to_postgis'

In [15]:
.to_sql(name='geo', con=engine)

AttributeError: 'GeometryDtype' object has no attribute 'base'

In [43]:
merged_df['Shortage_of_care_InfantsToddlers'] = merged_df['Shortage_of_care_InfantsToddlers'].astype(int)
merged_df['Shortage_of_Care_Preschoolers'] = merged_df['Shortage_of_Care_Preschoolers'].astype(int)
merged_df['full_shortage'] = merged_df['Shortage_of_Care_Preschoolers'] + merged_df['Shortage_of_care_InfantsToddlers']

In [55]:
town_geo_df = pd.read_csv('census_data/town_data.csv')

In [ ]:
town_geo_df.merge(town_level_df, how='inner', left_on='NAME', right_on='City').to_csv('town_data_map.csv')

In [57]:
DATA_DIR = '/Users/kylemagida/code/skylight/data-visualization/src/data_integration/unmet_needs'

In [1]:
DATA_DIR = '/Users/kylemagida/code/skylight/data-visualization/src/data_integration/unmet_needs'

In [7]:
town_level_df

,City,Children_Needing_Care_DemandEstimated_InfantsToddlers,Available_Spaces_for_Children_Supply_InfantsToddlers,Percent_of_Need_Met_InfantsToddlers,Shortage_of_care_InfantsToddlers,Children_Needing_Care_DemandEstimated_Preschoolers,Available_Spaces_for_Children_Supply_Preschoolers,Percent_of_Need_Met_Preschoolers,Shortage_of_Care_Preschoolers
0,Total,"74,600","23,585",32%,"-51,091","69,410","71,425",103%,"1,939"
1,Andover,5,5,100%,0,5,56,1120%,51
2,Ansonia,468,64,14%,-404,388,495,128%,107
3,Ashford,5,8,160%,3,5,62,1240%,57
4,Avon,351,217,62%,-134,234,529,226%,295
...,...,...,...,...,...,...,...,...,...
165,Windsor Locks,776,278,36%,-498,613,488,80%,-125
166,Wolcott,181,87,48%,-94,301,163,54%,-138
167,Woodbridge,5,80,1600%,75,5,192,3840%,187
168,Woodbury,263,49,19%,-214,204,101,50%,-103


In [8]:
town_level_funded

,City,Total_Infants_and_Toddlers_Needing_Care_InfantsToddlers,Infants_and_Toddlers_less_than_100%_FPL_needing_Care_InfantsToddlers,Infants_and_Toddlers_in_Families_Earning_Less_than_50%_State_Median_Income_Needing_Care_InfantsToddlers,Infants_and_Toddlers_in_Families_Earning_Less_than_State_Median_Income_Needing_Care_InfantsToddlers,Children_Served_by_Funding_estimated_InfantsToddlers,Funded Children-Children Needing Care/Subsidy_InfantsToddlers,Total_Preschoolers_Needing_Care_Preschoolers,Preschoolers_in_Poverty_Needing_Care_Preschoolers,Preschoolers_in_Families_Earning_Less_than_50%_State_Median_Income_Needing_Care_Preschoolers,Preschoolers_in_Families_Earning_Less_than_State_Median_Income_Needing_Care_Preschoolers,Children_Served_by_Funding_Estimate_Preschoolers,Funded_Children_Children_Needing_Funding_Preschoolers
0,Total,"74,600","12,670","24,524","42,182","10,641","-31,541","69,410","11,396","21,935","38,104","39,944","1,841"
1,Andover,5,*,*,*,0,-5,5,*,*,*,32,27
2,Ansonia,468,160,248,347,70,-277,388,133,205,288,548,260
3,Ashford,5,*,*,*,0,-5,5,*,*,*,49,44
4,Avon,351,15,28,76,24,-52,234,10,19,50,234,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Windsor,5,0,1,2,58,56,5,0,1,2,41,39
166,Wolcott,181,18,33,95,61,-34,301,30,54,157,122,-36
167,Woodbridge,5,*,*,*,30,25,5,*,*,*,31,26
168,Woodbury,263,7,20,106,2,-104,204,5,15,82,5,-77


In [10]:
puma_price_df = pd.read_csv('data/average_annual_advertised_price_by_age_setting_type_and_puma_region.csv')

In [12]:
puma_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   PUMA Region #                                   26 non-null     object
 1   Overall_Average_Rate_InfantsToddlers            26 non-null     object
 2   Centers_Average_Rate_InfantsToddlers            26 non-null     object
 3   Family Child Care Average Rate_InfantsToddlers  26 non-null     object
 4   Overall_Average_Rate_Preschoolers               26 non-null     object
 5   Centers_Average_Rate_Preschoolers               26 non-null     object
 6   Family_Child_Care_Average_Rate_Preschoolers     26 non-null     object
dtypes: object(7)
memory usage: 1.5+ KB
